# Ironbox: Memory System

This notebook demonstrates the Memory System in Ironbox, which allows the system to remember important information across sessions and provide context-aware responses.

## Overview

The Memory System in Ironbox consists of several components:

1. **Memory Manager**: Central component that coordinates memory operations
2. **Memory Indexer**: Processes and indexes new memories
3. **Memory Retriever**: Retrieves relevant memories based on semantic similarity
4. **Vector Store**: Manages vector embeddings for efficient semantic search
5. **Database**: Persistent storage for memories and embeddings

Let's explore how these components work together to provide a powerful memory system for Ironbox.

## Setup

First, let's import the necessary modules and initialize the Memory System.

In [ ]:
import os
import sys
from dotenv import load_dotenv
import numpy as np
from datetime import datetime

# Load environment variables from .env file
load_dotenv()

# Import Ironbox components
from ironbox.core.memory import MemorySystem, Memory, VectorStore
from ironbox.core.llm import LLMService
from ironbox.db.operations import DatabaseOperations

# Initialize the required services
db_ops = DatabaseOperations()
llm_service = LLMService()
vector_store = VectorStore(db_ops)
memory_system = MemorySystem(db_ops, llm_service, vector_store)

print("Memory System initialized successfully.")

## Storing Memories

Let's start by storing some memories in the system.

In [ ]:
# Function to store a memory
def store_memory(content, user_id="demo-user"):
    print(f"Storing memory: '{content}'")
    memory_id = memory_system.store_memory(content, user_id)
    print(f"Memory stored with ID: {memory_id}\n")
    return memory_id

# Store some memories about Kubernetes clusters
memories = [
    "My production Kubernetes cluster is hosted on GKE and has 5 nodes.",
    "The staging cluster is running Kubernetes version 1.25 and is hosted on AWS EKS.",
    "My database credentials are stored in the 'db-credentials' secret in the 'database' namespace.",
    "The frontend deployment in the production namespace has 3 replicas.",
    "I need to upgrade the development cluster to Kubernetes 1.26 next week."
]

memory_ids = []
for memory in memories:
    memory_id = store_memory(memory)
    memory_ids.append(memory_id)

## Memory Embedding Generation

When a memory is stored, the system generates an embedding vector using the LLM service. Let's examine how this works.

In [ ]:
# Function to demonstrate embedding generation
def demonstrate_embedding_generation(content):
    print(f"Generating embedding for: '{content}'")
    embedding = llm_service.generate_embedding(content)
    print(f"Embedding shape: {embedding.shape}")
    print(f"First 10 dimensions: {embedding[:10]}\n")
    return embedding

# Generate embeddings for a few example texts
example_texts = [
    "Kubernetes cluster on GKE",
    "Database credentials in secrets",
    "Frontend deployment with replicas"
]

for text in example_texts:
    demonstrate_embedding_generation(text)

## Retrieving Memories

Now let's retrieve memories based on semantic similarity to a query.

In [ ]:
# Function to retrieve memories
def retrieve_memories(query, user_id="demo-user", limit=3):
    print(f"Retrieving memories for query: '{query}'")
    memories = memory_system.retrieve_memories(query, user_id, limit=limit)
    print(f"Retrieved {len(memories)} memories:")
    for i, memory in enumerate(memories):
        print(f"  {i+1}. '{memory.content}' (Similarity: {memory.similarity:.4f})")
    print()
    return memories

# Retrieve memories for different queries
queries = [
    "Where is my production cluster hosted?",
    "What are my database credentials?",
    "How many replicas does the frontend have?",
    "What Kubernetes version is my staging cluster running?"
]

for query in queries:
    retrieve_memories(query)

## Memory Importance and Recency

The Memory System considers both importance and recency when retrieving memories. Let's explore how these factors affect memory retrieval.

In [ ]:
# Function to update memory importance
def update_memory_importance(memory_id, importance):
    print(f"Updating importance of memory {memory_id} to {importance}")
    memory_system.update_memory_importance(memory_id, importance)
    print("Importance updated successfully.\n")

# Update importance of some memories
update_memory_importance(memory_ids[0], 0.9)  # Production cluster is very important
update_memory_importance(memory_ids[2], 0.8)  # Database credentials are important

# Function to simulate memory access (updates last_accessed timestamp)
def access_memory(memory_id):
    print(f"Accessing memory {memory_id}")
    memory_system.access_memory(memory_id)
    print("Memory accessed successfully.\n")

# Access some memories to update their recency
access_memory(memory_ids[1])  # Staging cluster
access_memory(memory_ids[3])  # Frontend deployment

# Retrieve memories with importance and recency factors
print("Retrieving memories with importance and recency factors:")
query = "Tell me about my Kubernetes clusters"
memories = memory_system.retrieve_memories(query, "demo-user", limit=5, importance_weight=0.3, recency_weight=0.2)
print(f"Retrieved {len(memories)} memories:")
for i, memory in enumerate(memories):
    print(f"  {i+1}. '{memory.content}'")
    print(f"     Similarity: {memory.similarity:.4f}, Importance: {memory.importance:.4f}, Last Accessed: {memory.last_accessed}")

## Memory Context in Query Processing

Let's see how the Memory System provides context for query processing in the Agent Core.

In [ ]:
# Import Agent Core
from ironbox.core.agent_core import AgentCore

# Initialize Agent Core with Memory System
agent_core = AgentCore(memory_system=memory_system)

# Function to process a query with memory context
def process_query_with_memory(query, user_id="demo-user", session_id="demo-session"):
    print(f"Processing query with memory context: '{query}'")
    
    # Retrieve relevant memories
    memories = memory_system.retrieve_memories(query, user_id, limit=3)
    memory_context = "\n".join([f"- {memory.content}" for memory in memories])
    print(f"Memory context:\n{memory_context}\n")
    
    # Process the query with memory context
    response = agent_core.process_query(query, session_id=session_id, user_id=user_id)
    print(f"Response: {response['response']}\n")
    return response

# Process queries with memory context
memory_queries = [
    "What cloud provider hosts my production cluster?",
    "Where are my database credentials stored?",
    "Do I need to upgrade any of my clusters soon?"
]

for query in memory_queries:
    process_query_with_memory(query)

## Memory Summarization

The Memory System can summarize memories to provide a concise overview of what it knows about a topic.

In [ ]:
# Function to summarize memories
def summarize_memories(topic, user_id="demo-user"):
    print(f"Summarizing memories about: '{topic}'")
    
    # Retrieve relevant memories
    memories = memory_system.retrieve_memories(topic, user_id, limit=10)
    memory_texts = [memory.content for memory in memories]
    
    # Generate a summary using the LLM service
    prompt = f"Summarize the following information about {topic}:\n\n" + "\n".join([f"- {text}" for text in memory_texts])
    summary = llm_service.generate_text(prompt)
    
    print(f"Summary: {summary}\n")
    return summary

# Summarize memories about different topics
topics = ["Kubernetes clusters", "database information", "deployment configuration"]

for topic in topics:
    summarize_memories(topic)

## Memory Persistence

Let's examine how memories are stored in the database for persistence across sessions.

In [ ]:
# Function to demonstrate memory persistence
def demonstrate_memory_persistence():
    print("=== Memory Persistence Demo ===")
    
    # Store a new memory
    content = "The API server for my production cluster is exposed on port 6443"
    user_id = "demo-user"
    memory_id = memory_system.store_memory(content, user_id)
    print(f"Stored new memory with ID: {memory_id}")
    
    # Retrieve the memory from the database
    memory = db_ops.get_memory(memory_id)
    print(f"Retrieved from database: '{memory.content}'")
    print(f"User ID: {memory.user_id}")
    print(f"Created at: {memory.created_at}")
    print(f"Last accessed: {memory.last_accessed}")
    print(f"Importance: {memory.importance}")
    
    # Get the embedding
    embedding = db_ops.get_embedding(memory_id)
    print(f"Embedding dimensions: {len(embedding)}")
    print(f"First 5 dimensions: {embedding[:5]}")
    
    # Simulate a system restart
    print("\nSimulating system restart...")
    new_memory_system = MemorySystem(db_ops, llm_service, vector_store)
    
    # Retrieve the memory after restart
    query = "What port is my API server on?"
    memories = new_memory_system.retrieve_memories(query, user_id, limit=1)
    print(f"Retrieved after restart: '{memories[0].content}'")

# Run the demonstration
demonstrate_memory_persistence()

## Conclusion

In this notebook, we've explored the Memory System in Ironbox:

1. **Memory Storage**: Storing memories with content, user ID, and metadata
2. **Embedding Generation**: Creating vector embeddings for semantic search
3. **Memory Retrieval**: Finding relevant memories based on semantic similarity
4. **Importance and Recency**: Considering both factors in memory retrieval
5. **Memory Context**: Using memories to provide context for query processing
6. **Memory Summarization**: Generating concise summaries of related memories
7. **Memory Persistence**: Storing memories in the database for persistence across sessions

The Memory System is a critical component of Ironbox that allows it to maintain context across sessions and provide more personalized and relevant responses to user queries.